# **Linear Regression with Scikit-learn**

The goal is to fit a linear regression using `scikit-learn` on the same realistic 200-row dataset
we used earlier, and compare results to our `statsmodels` implementation.

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
import statsmodels.api as sm


## Recreate the same 200-row dataset (exactly as Notebook 3)

Let's recreate the same data so results are directly comparable across notebooks. This ensures differences reflect the implementation, not the data.


In [2]:
np.random.seed(42)
n = 200

# Predictors (same generation as Notebook 3)
age = np.random.randint(25, 81, size=n)                          
bmi = np.random.normal(loc=27, scale=3.5, size=n)                
bmi = np.clip(bmi, 18, 36)                                       
activity = np.random.poisson(lam=2.5, size=n)                    
activity = np.clip(activity, 0, 10)
salt = np.random.normal(loc=7.5, scale=2.0, size=n)              
salt = np.clip(salt, 3, 12)

# True coefficients (same)
intercept = 85.0
beta_age = 0.35
beta_bmi = 0.9
beta_activity = -1.2
beta_salt = 0.9

noise = np.random.normal(0, 7, size=n)                           

sbp = (intercept
       + beta_age * age
       + beta_bmi * bmi
       + beta_activity * activity
       + beta_salt * salt
       + noise)

df = pd.DataFrame({
    "Age": age,
    "BMI": np.round(bmi, 1),
    "Activity": activity,
    "SaltIntake": np.round(salt, 1),
    "SBP": np.round(sbp, 1)
})

# Quick check
df.head()


,Age,BMI,Activity,SaltIntake,SBP
0,63,25.9,2,7.4,131.3
1,76,29.4,2,8.0,153.8
2,53,26.1,5,10.6,122.0
3,39,25.7,4,5.5,111.7
4,67,31.5,4,9.5,142.1


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Age,200.0,51.8850,15.805723,25.0,38.000,52.00,66.000,79.0
BMI,200.0,27.4160,3.426228,18.0,25.175,27.25,30.025,36.0
Activity,200.0,2.4950,1.569167,0.0,1.000,2.00,3.000,8.0
SaltIntake,200.0,7.8485,1.888420,3.0,6.400,7.90,8.900,12.0
SBP,200.0,132.5290,9.994011,109.8,125.325,132.10,139.400,163.6


In [4]:
X = df[["Age", "BMI", "Activity", "SaltIntake"]]
y = df["SBP"]

# Simple train/test split for a generalization check
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
len(X_train), len(X_test)

(160, 40)

### What the train/test split is telling us?

This means our dataset is split as follows:
- 160 rows in training 
- 40 rows in testing

We're gonna fit the model on the training set and evaluate on the test set so we can get a simple measure of generalization.

## Fit the Linear Regression Model

In [5]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

### What happened when we called `fit()`?

When we called `lr.fit()` computed the best-fitting coefficients (and intercept) by minimizing mean squared error on the training data.

Sklearn hides the matrix math, but it actually performs the same OLS calculations (with numerical optimizations). Now we can inspect the learned parameters.

In [6]:
intercept_sklearn = lr.intercept_
coeff_sklearn = pd.Series(lr.coef_, index=X.columns)

print("Intercept (sklearn):", intercept_sklearn)
print(f"Coefficients (sklearn):\n{coeff_sklearn}")

Intercept (sklearn): 83.1924510108765
Coefficients (sklearn):
Age           0.345283
BMI           0.906111
Activity     -0.879531
SaltIntake    1.107268
dtype: float64


### Let's interpret intercept & coefficients

Our coefficients are approximately:
- Age `coef ≈ 0.35`
    - Each extra year of age → `+0.35 mmHg` higher SBP (controlling for BMI, activity, and salt intake).
- BMI `coef ≈ 0.91`
    - Each 1-unit increase in BMI → `+0.91 mmHg` higher SBP, holding the other variables constant.
- Activity `coef ≈ –0.88`
    - Each extra unit of activity (e.g., hour/week or activity-score point) → `~0.88 mmHg` lower SBP, on average.
- SaltIntake `coef ≈ 1.11`
    - Each 1-unit increase in SaltIntake → `+1.11 mmHg` higher SBP, after controlling for age, BMI, and activity.
- `Intercept ≈ 83.19`
    - This is our model’s baseline predicted SBP when Age, BMI, Activity, and SaltIntake are all zero (not meaningful by itself, but required for the linear model).

## Predictions on test set and RMSE

In [7]:
# Predictions on test set
y_pred_test = lr.predict(X_test)

# Compute RMSE
rmse_test = root_mean_squared_error(y_test, y_pred_test)
print("Test RMSE (sklearn):", rmse_test)

Test RMSE (sklearn): 7.423386349247889


* **RMSE** is measured in the same units as SBP (`mmHg`).
* Because RMSE ≈ the noise level we added (`~7 mmHg`), the model is performing about as well as the data-generating process allows.
* A smaller RMSE means better average prediction accuracy on the test set.
* Clinically, an RMSE of `~7 mmHg` means a typical prediction error of about `7 mmHg`, which is reasonable for population-level modeling but not sufficient for precise clinical decision-making.

Overall, our linear model predicts systolic blood pressure with errors of `±7–8 mmHg` on average, which is essentially the best achievable given the noise in the data. This indicates our model is behaving exactly as expected based on the data-generation process.

## Make Predictions for chosen Profiles

In [8]:
# Profiles similar to earlier notebooks
people = pd.DataFrame({
    "Age": [45, 60, 30],
    "BMI": [27, 31, 22],
    "Activity": [3, 1, 6],
    "SaltIntake": [8.0, 10.0, 5.0]
})

people_preds = lr.predict(people)
pd.DataFrame({"person": ["A (45y, 27)", "B (60y, 31)", "C (30y, 22)"],
              "predicted_SBP": np.round(people_preds, 1)})

,person,predicted_SBP
0,"A (45y, 27)",129.4
1,"B (60y, 31)",142.2
2,"C (30y, 22)",113.7


### Let's interpret the profile predictions

The predicted systolic blood pressures (SBP) for these profiles are:

| Person | Age | BMI | Activity | SaltIntake | Predicted SBP (`mmHg`) |
| ------ | --- | --- | -------- | ---------- | ---------------------- |
| A      | 45  | 27  | 3        | 8.0        | 129.4                  |
| B      | 60  | 31  | 1        | 10.0       | 142.2                  |
| C      | 30  | 22  | 6        | 5.0        | 113.7                  |


1. **Person A (45y, BMI 27, moderate activity, moderate salt) → `129.4 mmHg`**
   * Predicted SBP is in the upper range of normal.
   * Middle-aged with moderate BMI and activity → moderate SBP.

2. **Person B (60y, BMI 31, low activity, high salt) → `142.2 mmHg`**
   * Predicted SBP is elevated (hypertensive range).
   * Older age, higher BMI, low activity, and higher salt intake all contribute to higher predicted SBP.
   * This aligns with known risk factors for hypertension.

3. **Person C (30y, BMI 22, high activity, low salt) → `113.7 mmHg`**
   * Predicted SBP is in the healthy, normal range.
   * Younger age, lower BMI, high activity, and low salt intake → lowest SBP among the three.

## Compare `sklearn` coefficients to `statsmodels`

In [9]:
X_sm_full = sm.add_constant(X)  # add intercept
sm_full = sm.OLS(y, X_sm_full).fit()

comparison = pd.DataFrame({
    "statsmodels_coef": sm_full.params,
    "sklearn_coef": pd.Series([intercept_sklearn, *coeff_sklearn.values], index=["const"] + list(coeff_sklearn.index))
})

comparison

,statsmodels_coef,sklearn_coef
const,81.303274,83.192451
Age,0.378772,0.345283
BMI,0.917759,0.906111
Activity,-0.941351,-0.879531
SaltIntake,1.116203,1.107268


1. **Very similar results:**
   * All coefficients from `statsmodels` and `sklearn` are almost identical.
   * Small differences (intercept `81.3` vs `83.2`) are due to slight differences in how the two libraries handle the intercept and numerical precision.

2. **Signs and magnitudes match:**
   * Age and BMI → positive impact on SBP
   * Activity → negative impact (more activity → slightly lower SBP)
   * Salt intake → positive impact (more salt → higher SBP)

3. **Practical takeaway:**
   * Whether you use `statsmodels` or `sklearn`, your predictions and interpretations will be essentially the same.
   * SciPy's Statsmodels offers `p-values`, confidence intervals, and summary tables, which are useful for inference.
   * Sklearn is often used for prediction-focused workflows and integrates easily with pipelines and cross-validation.

**In short:** The two libraries produce nearly identical coefficients, and the small differences have negligible impact on predictions or interpretation.